#Require Lib

In [4]:
%%capture
# !pip install pydantic==2.4.2
!pip install setuptools
!pip install kaleido
!pip install cohere
!pip install openai
!pip install tiktoken
# !!apt install ffmpeg
# !pip install -q gradio==3.48.0
# !pip install -q transformers
!pip install -q librosa
!pip install -U whisper-ctranslate2
!sudo apt-get install libportaudio2
!pip install sounddevice
!pip install nemo_toolkit['tts'] soundfile

#ASR Model From HuggingFace🤗
Note : T4 GPU Required

In [1]:
from transformers import pipeline
import numpy as np

transcriber = pipeline("automatic-speech-recognition",
                       model="openai/whisper-tiny",
                       generate_kwargs={"task": "transcribe", "language": "Thai"})

#TTS From lunarlist

In [2]:
from nemo.collections.tts.models import UnivNetModel
from nemo.collections.tts.models import Tacotron2Model
import torch
import soundfile as sf

tacotron2model = Tacotron2Model.from_pretrained("lunarlist/tts-thai-last-step")
vcoder_model = UnivNetModel.from_pretrained(model_name="tts_en_libritts_univnet")

[NeMo W 2024-01-28 03:36:59 nemo_logging:393] c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
      warn(
    
[NeMo W 2024-01-28 03:37:02 nemo_logging:393] c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    


ModuleNotFoundError: No module named 'pyannote'

In [ ]:
def generate_speech(history,model=tacotron2model,vcoder_model=vcoder_model):
  text = history[-1][1]
  dict_idx={k:i for i,k in enumerate(model.hparams["cfg"]['labels'])}
  parsed2=torch.Tensor([[66]+[dict_idx[i] for i in text if i]+[67]]).int()
  spectrogram2 = model.generate_spectrogram(tokens=parsed2)
  audio2 = vcoder_model.convert_spectrogram_to_audio(spec=spectrogram2)

  # Save the audio to disk in a file called speech.wav
  sf.write("botspeech{0}.wav".format(len(history)), audio2.to('cpu').detach().numpy()[0], 22050)

  return '/content/'+"botspeech{0}.wav".format(len(history))

#UIs

In [ ]:
import gradio as gr
import time
import scipy.io.wavfile as wav
import numpy as np
import os

title = "Chatbot Demo"
DESCRIPTION = "This is a simplified chatbot demo without a specific model."
OTHER_HTML = ""

In [ ]:
def transcribe(voice_path,model=transcriber):
  return model(voice_path)

# Will be triggered on text submit (will send to generate_speech)
# def user_addtext(text,history):
#   history = [] if history is None else history
#   history = history + [[text, None]]
#   return gr.update(value="", interactive=False),history

# Will be triggered on voice submit (will transribe and send to generate_speech)
def add_file(voicefile_path,history):
  history = [] if history is None else history
  text = transcribe(voicefile_path)['text']
  history = history + [(text, None)]
  return text,history

# /tmp/gradio/f8590feaa311fc19fb95772ee79afda6fb498af5/audio.wav []
# สวัสดีครับ [('สวัสดีครับ', None)]

def bot_respond(history):
  history[-1][1] = ''
  user_current_txt = history[-1][0]
  bot_response_txt = user_current_txt
  for character in bot_response_txt:
    history[-1][1] += character
    time.sleep(0.1)
    yield history



In [ ]:
# Create a Gradio interface
with gr.Blocks(title=title) as demo:
    # gr.Markdown(DESCRIPTION)
    gr.Markdown(OTHER_HTML)

    chatbot = gr.Chatbot(
        [],
        elem_id="chatbot",
        bubble_full_width=False,
        avatar_images=('https://cdn-lfs.huggingface.co/repos/96/a2/96a2c8468c1546e660ac2609e49404b8588fcf5a748761fa72c154b2836b4c83/942cad1ccda905ac5a659dfd2d78b344fccfb84a8a3ac3721e08f488205638a0?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27hf-logo.svg%3B+filename%3D%22hf-logo.svg%22%3B&response-content-type=image%2Fsvg%2Bxml&Expires=1699706870&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5OTcwNjg3MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy85Ni9hMi85NmEyYzg0NjhjMTU0NmU2NjBhYzI2MDllNDk0MDRiODU4OGZjZjVhNzQ4NzYxZmE3MmMxNTRiMjgzNmI0YzgzLzk0MmNhZDFjY2RhOTA1YWM1YTY1OWRmZDJkNzhiMzQ0ZmNjZmI4NGE4YTNhYzM3MjFlMDhmNDg4MjA1NjM4YTA%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=U6mfsHJzPPvr0B3ZoR4SCeJcCYJjd8vo2VyVKSDA5Qm57-Mu8IRDwDIDmCSkWq1XacHA0Fq7owwjmH%7EPJiF1tuVXrqnmz3EFXFgh2090zh6k6dKthr4YLGPzixi0a6JbZLOspnjdr5sqXRR%7EHhQhAwcsgdkmMKYE%7ELrn-AVkmGIDdLl%7EE5ebMsUsfA2hEleRfElavgGthViGGjHpM1y70uo1cftc5wW0cRpfixEGQ3Nr2RYVs8UeP6eC33bbrqqBfTniEj-FOJ7XyQCtKTy5zOu-mLAdRoAzVOn%7Es-X6OKhGn9Bd%7EK6bfHuoEMDfW70RCcUpB4O3SwqtNRxIfvmYug__&Key-Pair-Id=KVTP0A1DKRTAX','https://avatars.githubusercontent.com/u/51063788?s=200&v=4')
    )
    with gr.Row():
        voice_btn = gr.Audio(sources="microphone",type="filepath", scale=4, streaming=True)
        txt = gr.Textbox(
            scale=3,
            show_label=False,
            placeholder="Enter text and press enter, or speak to your microphone",
            container=False,
            interactive=True,
        )
        #txt_btn = gr.Button(value="Submit text", scale=1)

    with gr.Row():
        sentence = gr.Textbox(visible=False)
        audio = gr.Audio(
            value=None,
            label="Generated audio response",
            streaming=True,
            autoplay=True,
            interactive=False,
            show_label=True,
        )

    clear_btn = gr.ClearButton([chatbot, audio,txt])

    voice_msg =  voice_btn.stop_recording(add_file, [voice_btn,chatbot], [txt,chatbot], queue=False).then(bot_respond,chatbot,chatbot).then(generate_speech,chatbot,audio)

    clear_btn.click(lambda: None,None,None, None, [chatbot,audio,txt], queue=False)

demo.queue().launch(debug=True)

[NeMo W 2023-11-25 13:00:41 nemo_logging:349] <ipython-input-7-d9404ee0b601>:13: GradioUnusedKwargWarning: You have unused kwarg parameters in Audio, please remove them: {'sources': 'microphone'}
      voice_btn = gr.Audio(sources="microphone",type="filepath", scale=4, streaming=True)
    
[NeMo W 2023-11-25 13:00:41 nemo_logging:349] <ipython-input-7-d9404ee0b601>:38: GradioDeprecationWarning: The 'status_tracker' parameter has been deprecated and has no effect.
      clear_btn.click(lambda: None,None,None, None, [chatbot,audio,txt], queue=False)
    


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://64429502b10014937f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://64429502b10014937f.gradio.live
